# 1. Import Libraries and Define Paths


In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from mtcnn import MTCNN
from keras_facenet import FaceNet
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import StratifiedKFold, train_test_split, GridSearchCV, StratifiedShuffleSplit
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from collections import Counter

# Define paths
DATA_DIR = "C:/Users/DELL/Downloads/TASKS/TASK1/DATA"


# 2. Function to Load Images from Directory


In [2]:
# Function to load images from directory
def load_images_from_dir(dir_path, label):
    images = []
    labels = []
    for filename in os.listdir(dir_path):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            img_path = os.path.join(dir_path, filename)
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
            images.append(img)
            labels.append(label)
    return images, labels


# 3. Function to Load All Images


In [3]:
# Function to load all images
def load_all_images(data_dir):
    actor_dirs = ["Elon_Musk", "Angelina_Jolie", "Stromae"]
    images = []
    labels = []

    for actor in actor_dirs:
        dir_path = os.path.join(data_dir, actor)
        imgs, lbls = load_images_from_dir(dir_path, actor)
        images.extend(imgs)
        labels.extend(lbls)

    return np.array(images), np.array(labels)


# 4. Initialize FaceNet and Get Embeddings


In [4]:
# Initialize FaceNet
embedder = FaceNet()

# Function to get embeddings
def get_embedding(face_img, embedder):
    face_img = face_img.astype('float32')
    face_img = np.expand_dims(face_img, axis=0)
    yhat = embedder.embeddings(face_img)
    return yhat[0]


# 5. Load Data and Compute Embeddings


In [5]:
# Load all images
X, Y = load_all_images(DATA_DIR)

# Calculate embeddings
EMBEDDED_X = [get_embedding(img, embedder) for img in X]
EMBEDDED_X = np.asarray(EMBEDDED_X)

# Encode labels
encoder = LabelEncoder()
Y_encoded = encoder.fit_transform(Y)

# Flatten data
EMBEDDED_X_flat = EMBEDDED_X.reshape(EMBEDDED_X.shape[0], -1)


C:\Users\DELL\AppData\Local\Temp/ipykernel_22256/723465572.py:13: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(images), np.array(labels)


1/1 [==============================] - 0s 45ms/step


# 6. Define Parameter Grid for GridSearchCV


In [6]:
# Define parameter grid for GridSearchCV
param_grid = {
    'svc1__C': [0.1, 0.5, 1.0],
    'svc2__C': [0.1, 0.5, 1.0],
    'svc2__gamma': [0.001, 0.01, 0.1]
}

# Initialize models with different parameters
model1 = SVC(kernel='linear', probability=True)
model2 = SVC(kernel='rbf', probability=True)

ensemble_model = VotingClassifier(estimators=[('svc1', model1), ('svc2', model2)], voting='soft')

grid_search = GridSearchCV(estimator=ensemble_model, param_grid=param_grid, cv=3)


# 7. Stratified K-Fold Cross Validation


In [10]:
# Stratified K-Fold Cross Validation with more folds
kf = StratifiedKFold(n_splits=7)  # Increase the number of splits if needed
train_accuracies = []
val_accuracies = []
test_accuracies = []

def check_distribution(labels, title):
    class_distribution = Counter(labels)
    print(f"Distribution des classes dans {title}:")
    for label, count in class_distribution.items():
        print(f"{label}: {count}")

def plot_confusion_matrix(y_true, y_pred, labels, title):
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
    plt.figure(figsize=(8, 6))
    disp.plot(cmap=plt.cm.Blues)
    plt.title(title)
    plt.show()

# Loop through folds
for train_index, val_test_index in kf.split(EMBEDDED_X_flat, Y_encoded):
    X_train, X_temp = EMBEDDED_X_flat[train_index], EMBEDDED_X_flat[val_test_index]
    y_train, y_temp = Y_encoded[train_index], Y_encoded[val_test_index]
   
    # Further split temp set into validation and test sets with increased size
    sss = StratifiedShuffleSplit(n_splits=1, test_size=0.55, random_state=75)  # Increase test size
    for val_index, test_index in sss.split(X_temp, y_temp):
        X_val, X_test = X_temp[val_index], X_temp[test_index]
        y_val, y_test = y_temp[val_index], y_temp[test_index]

    
    # Train model
    grid_search.fit(X_train, y_train)

    # Evaluate performance
    ypreds_train = grid_search.best_estimator_.predict(X_train)
    ypreds_val = grid_search.best_estimator_.predict(X_val)
    ypreds_test = grid_search.best_estimator_.predict(X_test)


# 8. Real-Time Face Detection and Recognition


In [ ]:
import os
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from mtcnn import MTCNN
from keras_facenet import FaceNet
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import pickle
from keras.preprocessing.image import ImageDataGenerator

HAARCASCADE_PATH = "C:/Users/DELL/Downloads/TASKS/TASK1/haarcascade_frontalface_default.xml"
SIMILARITY_THRESHOLD = 0.6

cap = cv.VideoCapture(0)
haarcascade = cv.CascadeClassifier(HAARCASCADE_PATH)
facenet = FaceNet()

while cap.isOpened():
    _, frame = cap.read()
    rgb_img = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    gray_img = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    faces = haarcascade.detectMultiScale(gray_img, 1.3, 5)

    for (x, y, w, h) in faces:
        face = rgb_img[y:y + h, x:x + w]
        face = cv.resize(face, (160, 160))
        img = np.expand_dims(face, axis=0)

        # Get embeddings for the face
        ypred = facenet.embeddings(img)

        # Calculate similarity with known faces
        similarities = cosine_similarity(ypred, EMBEDDED_X)
        max_similarity = np.max(similarities)

        # Determine label based on similarity
        if max_similarity < SIMILARITY_THRESHOLD:
            final_name = 'unknown'
        else:
            best_match_index = np.argmax(similarities)
            final_name = Y[best_match_index]

        # Draw rectangle and label on the face
        cv.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
        cv.putText(frame, str(final_name), (x, y - 10), cv.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2, cv.LINE_AA)

    # Display the frame
    cv.imshow('frame', frame)

    # Exit loop if 'q' is pressed
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

# Release capture and destroy all windows
cap.release()
cv.destroyAllWindows()
